# 📓 Example: Using the Energy Mix Optimizer
This notebook demonstrates how to forecast renewable generation and optimise the energy mix.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.utils import predict_generation, optimise_mix

## 1. Load dataset

In [ ]:
data = pd.read_csv('data/energy_mix_dataset.csv', parse_dates=['datetime'], index_col='datetime')
features = data.drop(columns=['demand', 'solar_pv', 'wind', 'hydro'])
y_solar = data['solar_pv']
y_wind = data['wind']

## 2. Define tomorrow's weather & price forecast

In [ ]:
tomorrow = pd.DataFrame([{
    "surface_solar_radiation_downwards": 240,
    "10m_u_component_of_wind": 3.0,
    "10m_v_component_of_wind": 1.0,
    "2m_temperature": 296,
    "hour": 13,
    "month": 6,
    "day_of_week": 2,
    "is_weekend": 0,
    "is_holiday": 0,
    "price": 70
}])

## 3. Predict renewable generation

In [ ]:
solar_pred = predict_generation(features, y_solar, tomorrow)
wind_pred = predict_generation(features, y_wind, tomorrow)
hydro_pred = data['hydro'].mean()
print(f'Solar: {solar_pred:.2f} MWh | Wind: {wind_pred:.2f} MWh | Hydro: {hydro_pred:.2f} MWh')

## 4. Optimise mix for a given demand

In [ ]:
demand = 580
mix, total_cost, total_emissions, success = optimise_mix(solar_pred, wind_pred, hydro_pred, demand)
if success:
    technologies = ["Solar", "Wind", "Hydro", "Gas", "Nuclear"]
    for tech, val in zip(technologies, mix):
        print(f'{tech}: {val:.2f} MWh')
    print(f'\nTotal Cost: €{total_cost:.2f}')
    print(f'Total Emissions: {total_emissions:.2f} tCO₂')
else:
    print("Optimization failed.")

## 5. Visualise the Optimised Mix

In [ ]:
if success:
    technologies = ["Solar", "Wind", "Hydro", "Gas", "Nuclear"]
    plt.figure(figsize=(8, 5))
    plt.bar(technologies, mix, color='skyblue')
    plt.title('Optimised Energy Mix')
    plt.ylabel('Dispatch (MWh)')
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()